In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook

## Data Sampling

#### Mobile Log data

In [2]:
# Mobile
mobile = pd.read_csv("./data(다운로드금지)/PCMB공통수집패널로그_MB_201406.csv", low_memory=False, header=None)

In [3]:
mobile.columns = ["UID","Full_URL","Site","Domain","Time", "Title",
              "keyword_p","keyword_t",
              "category_1","category_2","category_3","category_4","category_5","category_6",
              "ownership_1","ownership_2"]

In [4]:
mobile.UID = mobile.UID.map(lambda x: x.replace(" ",""))

#### PC Log data

In [5]:
# PC
pc1 = pd.read_csv("/home/da/jupyter/김현준/PCMB공통수집패널로그_PC_20140601_10.csv", header=None, low_memory=False)
pc2 = pd.read_csv("/home/da/jupyter/김현준/PCMB공통수집패널로그_PC_20140611_20.csv", header=None, low_memory=False)
pc3 = pd.read_csv("/home/da/jupyter/김현준/PCMB공통수집패널로그_PC_20140621_30.csv", header=None, low_memory=False)

In [6]:
pc1.columns = ["UID","Full_URL","Site","Domain","Time", "Title",
              "keyword_p","keyword_t",
              "category_1","category_2","category_3","category_4","category_5","category_6",
              "ownership_1","ownership_2"]
pc2.columns = ["UID","Full_URL","Site","Domain","Time", "Title",
              "keyword_p","keyword_t",
              "category_1","category_2","category_3","category_4","category_5","category_6",
              "ownership_1","ownership_2"]
pc3.columns = ["UID","Full_URL","Site","Domain","Time", "Title",
              "keyword_p","keyword_t",
              "category_1","category_2","category_3","category_4","category_5","category_6",
              "ownership_1","ownership_2"]

In [7]:
pc1.UID = pc1.UID.map(lambda x: x.replace(" ",""))
pc2.UID = pc2.UID.map(lambda x: x.replace(" ",""))
pc3.UID = pc3.UID.map(lambda x: x.replace(" ",""))

In [8]:
pc = pd.concat([pc1, pc2, pc3], axis=0)

#### Survey data

In [9]:
# 패널 서베이
survey = pd.read_excel("./data(다운로드금지)/140716_SSK 구매행태 조사 Raw Data_F.xlsx")
survey.UID = survey.UID.map(lambda x: x.replace(" ",""))
print("서베이의 총 UID 개수:", survey.UID.nunique())

서베이의 총 UID 개수: 1066


In [10]:
uid_list = survey.UID.unique().tolist()

In [12]:
# Mobile/PC 여부 컬럼 추가
# 1 = PC
# 0 = Mobile
mobile["PC"] = 0
pc["PC"] = 1

### Mobile/PC to one dataframe

In [13]:
data = pd.concat([mobile, pc], axis=0)

In [14]:
data.reset_index(drop=True, inplace=True)

In [16]:
# Time 데이터 속성 변경
data["Time"] = pd.to_datetime(data["Time"])

In [17]:
# 안쓰는 컬럼 삭제
data.drop(["Site","Title", "keyword_t",
           "category_3", "category_4", "category_5", "category_6"],
           axis=1, inplace=True)

In [18]:
print("# of rows:", len(data))

# of rows: 50999038


In [19]:
print("# of unique UIDs:", data.UID.nunique())

# of unique UIDs: 2103


### 서베이에 참여한 패널들만 sample

In [20]:
sample = data[data["UID"].isin(uid_list)].copy()

In [21]:
sample.reset_index(drop=True, inplace=True)

In [22]:
sample.head()

,UID,Full_URL,Domain,Time,category_1,category_2,ownership_1,ownership_2,PC
0,20130820A574A4005D61,http://www.koreapas.com/m/view.php?id=pashion&...,koreapas.com/,2014-06-01 22:26:59,NaN,NaN,NaN,NaN,0
1,20130923D5653FF28FD9,http://www.swimdoctormall.co.kr/?mode=PC,swimdoctormall.co.kr/,2014-06-05 21:59:00,NaN,NaN,NaN,NaN,0
2,20130820A574A4005D61,http://www.koreapas.com/m/view.php?id=pashion&...,koreapas.com/,2014-06-01 22:27:28,NaN,NaN,NaN,NaN,0
3,20130923D5653FF28FD9,http://www.swimdoctormall.co.kr/mall/index.php,swimdoctormall.co.kr/,2014-06-05 21:59:09,NaN,NaN,NaN,NaN,0
4,20130923D5653FF28FD9,http://www.swimdoctor.com/bbs/bbs_view.php?cod...,swimdoctor.com/,2014-06-06 02:40:15,NaN,NaN,NaN,NaN,0


In [23]:
# 데이터프레임 저장
sample.to_csv("./data(다운로드금지)/0222_PC&Mob_06.csv", index=False)

In [ ]:
# 저장한 데이터프레임 불러오기
original = pd.read_csv("./data(다운로드금지)/0222_PC&Mob_06.csv")

In [45]:
original["Time"] = pd.to_datetime(original["Time"])

In [155]:
data = original.copy()

## Domain

Ex: http://m.enews24.interest.me/news/article.asp?...   
URL이 그대로 Domain에 들어온 경우가 있음 -> http://, https://, m., www. 으로 시작됨   
위의 부분들을 잘라내고, slash 기준 앞쪽 url만 Domain으로 사용
기존 Domain들에 있던 마지막 slash 삭제   
To check, data2[data2["Domain"].str.startswith("https://")]   

In [156]:
def prep_domain(domain):
    
    prep_domain = domain.replace("http://m.","")
    prep_domain = prep_domain.replace("http://www.","")
    prep_domain = prep_domain.replace("https://m.","")
    prep_domain = prep_domain.replace("https://www.","")
    prep_domain = prep_domain.replace("http://","")
    prep_domain = prep_domain.replace("https://","")
    prep_domain = prep_domain.split("/")[0]
    prep_domain = prep_domain.split(":")[0]
    prep_domain = prep_domain.split("ref=")[-1]
    
    return prep_domain

In [157]:
data["Domain"] = data["Domain"].apply(lambda x: prep_domain(x))

전처리 이후, Domain이 공백인 row를 찾으면 Full_URL이 http:/// 인 row가 4개 있음   
4개 row 삭제

In [158]:
data.drop(data[data["Domain"]==""].index, axis=0, inplace=True)

## ownership_1, ownership_2

### 결측행 최대한 채워넣기

In [164]:
# zum.com
# 47039 rows
# Portal, 줌
# print(data[data.Domain.str.startswith("zum.com")])
data["ownership_1"] = np.where(data["Domain"]=="zum.com", "Portal", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="zum.com", "줌", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="zum.com", "컴퓨터/인터넷", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="zum.com", "인터넷", data["category_2"])

In [165]:
# daum.net
# 4562
# Portal, Daum
data["ownership_1"] = np.where(data["Domain"]=="daum.net", "Portal", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="daum.net", "Daum", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="daum.net", "컴퓨터/인터넷", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="daum.net", "인터넷", data["category_2"])

In [166]:
# swimdoctormall.co.kr
# swimdoctor.com
# Retailer, 스포츠/레저
data["ownership_1"] = np.where(data["Domain"]=="swimdoctormall.co.kr", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="swimdoctormall.co.kr", "스포츠/레저", data["ownership_2"])
data["ownership_1"] = np.where(data["Domain"]=="swimdoctormall.com", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="swimdoctormall.com", "스포츠/레저", data["ownership_2"])
data["ownership_1"] = np.where(data["Domain"]=="swimdoctor.com", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="swimdoctor.com", "스포츠/레저", data["ownership_2"])

data["category_1"] = np.where(data["Domain"]=="swimdoctormall.co.kr", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="swimdoctormall.co.kr", "스포츠/레저", data["category_2"])
data["category_1"] = np.where(data["Domain"]=="swimdoctormall.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="swimdoctormall.com", "스포츠/레저", data["category_2"])
data["category_1"] = np.where(data["Domain"]=="swimdoctor.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="swimdoctor.com", "스포츠/레저", data["category_2"])

In [167]:
# mazia.kr
# Brand, 의류/잡화
data["ownership_1"] = np.where(data["Domain"]=="mazia.kr", "Brand", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="mazia.kr", "의류/잡화", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="mazia.kr", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="mazia.kr", "의류/잡화", data["category_2"])

In [168]:
# gundamart.co.kr
# Retailer, 취미
data["ownership_1"] = np.where(data["Domain"]=="gundamart.co.kr", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="gundamart.co.kr", "취미", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="gundamart.co.kr", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="gundamart.co.kr", "취미", data["category_2"])

In [169]:
# ssteamboy.com
# Maker, 전자제품
data["ownership_1"] = np.where(data["Domain"]=="ssteamboy.com", "Maker", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="ssteamboy.com", "전자제품", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="ssteamboy.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="ssteamboy.com", "전기/전자", data["category_2"])

In [170]:
# caromshop.co.kr
# Retailer, 스포츠/레저
data["ownership_1"] = np.where(data["Domain"]=="caromshop.co.kr", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="caromshop.co.kr", "스포츠/레저", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="caromshop.co.kr", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="caromshop.co.kr", "스포츠/레저", data["category_2"])

In [171]:
# gseshop.co.kr
# Retailer, 종합쇼핑몰
data["ownership_1"] = np.where(data["Domain"]=="gseshop.co.kr", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="gseshop.co.kr", "종합쇼핑몰", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="gseshop.co.kr", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="gseshop.co.kr", "종합쇼핑몰", data["category_2"])

In [172]:
# angel-dshop.com
# Maker, 선물
data["ownership_1"] = np.where(data["Domain"]=="angel-dshop.com", "Maker", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="angel-dshop.com", "선물", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="angel-dshop.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="angel-dshop.com", "선물", data["category_2"])

In [173]:
# enshop.kr
# Retailer, 스포츠/레저
data["ownership_1"] = np.where(data["Domain"]=="enshop.kr", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="enshop.kr", "스포츠/레저", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="enshop.kr", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="enshop.kr", "스포츠/레저", data["category_2"])

In [174]:
# billiardshop.kr
# Retailer, 스포츠/레저
data["ownership_1"] = np.where(data["Domain"]=="billiardshop.kr", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="billiardshop.kr", "스포츠/레저", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="billiardshop.kr", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="billiardshop.kr", "스포츠/레저", data["category_2"])

In [175]:
# jungoneshop.com
# Maker, 식품/음료
data["ownership_1"] = np.where(data["Domain"]=="jungoneshop.com", "Maker", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="jungoneshop.com", "식품/음료", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="jungoneshop.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="jungoneshop.com", "식품/음료", data["category_2"])

In [176]:
# nseshop.com 
# Retailer, 홈쇼핑
data["ownership_1"] = np.where(data["Domain"]=="nseshop.com", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="nseshop.com", "홈쇼핑", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="nseshop.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="nseshop.com", "종합쇼핑몰", data["category_2"])

In [177]:
# shop.olleh.com
data["ownership_1"] = np.where(data["Domain"]=="shop.olleh.com", "Maker", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="shop.olleh.com", "무선/이동통신", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="shop.olleh.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="shop.olleh.com", "통신/네트워크", data["category_2"])

In [178]:
# shop7989.com
data["ownership_1"] = np.where(data["Domain"]=="shop7989.com", "Brand", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="shop7989.com", "의류/잡화", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="shop7989.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="shop7989.com", "의류/잡화", data["category_2"])

In [179]:
# jstyleshop.net
data["ownership_1"] = np.where(data["Domain"]=="jstyleshop.net", "Brand", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="jstyleshop.net", "의류/잡화", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="jstyleshop.net", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="jstyleshop.net", "의류/잡화", data["category_2"])

In [180]:
# bearpawshop.com
data["ownership_1"] = np.where(data["Domain"]=="bearpawshop.com", "Brand", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="bearpawshop.com", "의류/잡화", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="bearpawshop.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="bearpawshop.com", "의류/잡화", data["category_2"])

In [181]:
# bongjashop.com
data["ownership_1"] = np.where(data["Domain"]=="bongjashop.com", "Brand", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="bongjashop.com", "의류/잡화", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="bongjashop.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="bongjashop.com", "의류/잡화", data["category_2"])

In [182]:
# inurishop.com
data["ownership_1"] = np.where(data["Domain"]=="inurishop.com", "Maker", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="inurishop.com", "식품/음료", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="inurishop.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="inurishop.com", "식품/음료", data["category_2"])

In [183]:
# jogunshop.com
data["ownership_1"] = np.where(data["Domain"]=="jogunshop.com", "Brand", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="jogunshop.com", "의류/잡화", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="jogunshop.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="jogunshop.com", "의류/잡화", data["category_2"])

In [184]:
# hanashop.com
data["ownership_1"] = np.where(data["Domain"]=="hanashop.com", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="hanashop.com", "여행사", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="hanashop.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="hanashop.com", "종합쇼핑몰", data["category_2"])

In [185]:
# nubizio.co.kr
data["ownership_1"] = np.where(data["Domain"]=="nubizio.co.kr", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="nubizio.co.kr", "홈인테리어", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="nubizio.co.kr", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="nubizio.co.kr", "가정/생활용품", data["category_2"])

In [186]:
# jomashop.com
data["ownership_1"] = np.where(data["Domain"]=="jomashop.com", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="jomashop.com", "의류/잡화", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="jomashop.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="jomashop.com", "의류/잡화", data["category_2"])

In [187]:
# gsshop.com
data["ownership_1"] = np.where(data["Domain"]=="gsshop.com", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="gsshop.com", "홈쇼핑", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="gsshop.com", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="gsshop.com", "종합쇼핑몰", data["category_2"])

In [188]:
# janeshopping.co.kr
data["ownership_1"] = np.where(data["Domain"]=="janeshopping.co.kr", "Retailer", data["ownership_1"])
data["ownership_2"] = np.where(data["Domain"]=="janeshopping.co.kr", "의류/잡화", data["ownership_2"])
data["category_1"] = np.where(data["Domain"]=="janeshopping.co.kr", "비즈니스/쇼핑", data["category_1"])
data["category_2"] = np.where(data["Domain"]=="janeshopping.co.kr", "의류/잡화", data["category_2"])

### 나머지 결측행 값, '기타'로 넣기(혹은 결측행 제거)

In [189]:
len(data[data["ownership_1"].isnull()])

8765433

In [190]:
len(data[data["ownership_2"].isnull()])

8765433

In [191]:
len(data[data["category_1"].isnull()])

8765433

In [192]:
len(data[data["category_2"].isnull()])

8765433

In [193]:
# data["ownership_1"].fillna("기타", inplace=True)
# data["ownership_2"].fillna("기타", inplace=True)

# data["ownership_1"].fillna("기타", inplace=True)
# data["ownership_2"].fillna("기타", inplace=True)

# data["category_1"].fillna("기타", inplace=True)
# data["category_2"].fillna("기타", inplace=True)

In [19]:
# 그 외 결측치있는 row 제거
# data.dropna(subset=["ownership_1"], axis=0, inplace=True)

## Session_ID 추가

In [194]:
data.sort_values(by=["UID","Time"], inplace=True)

In [195]:
cond1 = data.Time - data.Time.shift(1) > pd.Timedelta(11, 'm')
cond2 = data.UID != data.UID.shift(1)
data['Session_ID'] = (cond1|cond2).cumsum()

## 사이트의 메인페이지 로그기록 제거

In [199]:
def remove_portal_site(data):
    portal = ['naver.com', 'google.co.kr', 'daum.net', 'nate.com', 'naver.com', 'zum.com']
    portal_index= []
    for UID in tqdm_notebook(data.UID.unique()):
        temp = data[data.UID == UID]
        for session in temp.Session_ID.unique():
            temp1 = temp[temp['Session_ID'] == session]
            temp1 = temp1.reset_index()
            if temp1['Domain'][0] not in portal:
                continue
            else:
                for i in range(len(temp1['Domain'])-1):
                    if temp1['Domain'][i] != temp1['Domain'][i+1]:
                        break
                    else:
                        portal_index.append(int(temp1['index'][i]))
    data.drop(portal_index, inplace=True)
    return data

In [200]:
data2 = remove_portal_site(data)

# **아래는 이외 변수 추가를 위한 내용**

## 파생 변수 추가

### 세션 당 소요시간

In [201]:
se_time_min = data2.groupby(['UID', 'Session_ID'])['Time'].min()
se_time_max = data2.groupby(['UID', 'Session_ID'])['Time'].max()

In [ ]:
pd.DataFrame(se_time_max-se_time_min)

### 세션 당 방문한 사이트 수

### 세션 별 쇼핑 관련 사이트 방문 수